### Riddler Express:

I have three dice (d4, d6, d8) on my desk that I fiddle with while working, much to the chagrin of my co-workers. For the uninitiated, the d4 is a tetrahedron that is equally likely to land on any of its four faces (numbered 1 through 4), the d6 is a cube that is equally likely to land on any of its six faces (numbered 1 through 6), and the d8 is an octahedron that is equally likely to land on any of its eight faces (numbered 1 through 8).

I like to play a game in which I roll all three dice in “numerical” order: d4, then d6 and then d8. I win this game when the three rolls form a strictly increasing sequence (such as 2-4-7, but not 2-4-4). What is my probability of winning?


#### Analytical Solution:

*Solution 1: Determining All Combinations*

Determine all possible sequences that adhere relative to total combinations
 
Total Combinations: `4*6*8` = 192
Strictly increasing sequences: 48
- 1: 20
    - 2 -> {3-8} = 6
    - 3 -> {4-8} = 5
    - 4 -> {5-8} = 4
    - 5 -> {6-8} = 3
    - 6 -> {7-8} = 2
- 2: 14
    - 3 -> {4-8} = 5
    - 4 -> {5-8} = 4
    - 5 -> {6-8} = 3
    - 6 -> {7-8} = 2
- 3: 9
    - 4 -> {5-8} = 4
    - 5 -> {6-8} = 3
    - 6 -> {7-8} = 2
- 4: 5
    - 5 -> {6-8} = 3
    - 6 -> {7-8} = 2
    
 Solution: `48/192 = 0.25`

### Python Solution

This could be massively sped up using numpy, which I will do next. But for now just creating a simple class which can store a single game and determine if strictly increasing or not. 

In [1]:
# sample test
test = [3,4,5]
print([test[i] < test[i+1] for i in range(len(test)-1)])

# all this will fail at the first element, so bit faster than doing all checks
# try out all on strictly increasing
output = all([test[i] < test[i+1] for i in range(len(test)-1)])
print(output)

# try out all on a non-strictly increasing
test = [9,4,5]
output = all([test[i] < test[i+1] for i in range(len(test)-1)])
print(output)

[True, True]
True
False


In [2]:
import numpy as np 
import time

class multiDie():
    def __init__(self, sides):
        self.sides = sides
        
    def checkRolls(self):
        """Check if strictly increasing"""
        # roll all
        self._buildSeq()
        
        # use all to check if each element is greater than prior idx
        output = all([self.rolls[i] < self.rolls[i+1] for i in range(len(self.rolls)-1)])
        return output
        
    def _buildSeq(self):
        """Roll len(sides) die of varying sizes"""
        self.rolls = [self._rollN(sides) for sides in self.sides]
    
    def _rollN(self, n):
        """Roll a single die with N sides"""
        return np.random.choice(n) + 1

In [3]:
# try some runs -> in testing i did confirm strictly increasing is captured
# but didn't set a seed (oops) -> should have an option in class to set a sequence

# 1
tester = multiDie([4,6,8])
output = tester.checkRolls()
print(tester.rolls)
print(output)

# 2
tester = multiDie([4,6,8])
output = tester.checkRolls()
print(tester.rolls)
print(output)

# 3
tester = multiDie([4,6,8])
output = tester.checkRolls()
print(tester.rolls)
print(output)

[2, 4, 2]
False
[4, 4, 5]
False
[4, 6, 4]
False


In [4]:
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000]

for sim in sim_list:    
    start = time.time()
    counter = 0
    for _ in range(sim):
        tester = multiDie([4,6,8])
        counter += tester.checkRolls()
    print(f"Sim of {sim} got probability of {counter/sim}")
    print(f"Total time: {time.time() - start:2f} seconds")

Sim of 100 got probability of 0.25
Total time: 0.002213 seconds
Sim of 1000 got probability of 0.251
Total time: 0.019030 seconds
Sim of 10000 got probability of 0.2488
Total time: 0.149845 seconds
Sim of 100000 got probability of 0.25171
Total time: 1.307051 seconds
Sim of 1000000 got probability of 0.250216
Total time: 12.916352 seconds


### Numpy Implementation 

Much faster! `numpy` allows us to get very high sim results without slowing down. 

Approach:

- Build a large n-dim array where `n` = total dies in play
- Fill in each dimension of the n-dim array as a different die's sample rolls 
    - This means a single row would be results of `{4d, 6d, 8d}'
    - Also makes it easy to quickly expand to Extra Credit problem
- `np.diff`: Calculate the n-th discrete difference along the given axis.
    - This will compare each value against the value at the next idx
    - E.g. `(4,5,16)` -> `(1,11)`
    - A strictly increasing sequence will always have values > 0
- find the sum of all rows where `np.diff` had a min > 0

In [5]:
sim = 100
die_list = [4,6,8]

rolls = np.zeros(shape=(sim,len(die_list))) # build empty np array to store sim x total die

# rolls over each die type
for i, n in enumerate(die_list):
    rolls[:,i] = np.random.choice(n, sim) + 1 # do +1 to account for 0 index

# assert we have proper rolls -> max val must be <= sides of die
for i, n in enumerate(die_list):
    assert(np.max(rolls[:,i]) <= n)
    
# use np.diff to create array of diffs of 0th -> 1st, 1st -> 2nd, etc.
# min for strictly increasing is 1
min_diff = np.min(np.diff(rolls), axis = 1) # axis = 1 indicates row
np.sum(min_diff > 0 ) # represents strictly increasing

32

In [6]:
# run larger sim: 
die_list = [4,6,8]
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000, 5_000_000]

for sim in sim_list:
    start = time.time()
    rolls = np.zeros(shape=(sim,len(die_list))) # build empty np array to store sim x total die

    # rolls over each die type
    for i, n in enumerate(die_list):
        rolls[:,i] = np.random.choice(n, sim) + 1 # do +1 to account for 0 index

    # use np.diff to create array of diffs of 0th -> 1st, 1st -> 2nd, etc.
    min_diff = np.min(np.diff(rolls), axis = 1) # axis = 1 indicates row
    print(f"Sim of {sim} got probability of {np.sum(min_diff > 0 )/sim}")
    print(f"Total time: {time.time() - start:2f} seconds")

Sim of 100 got probability of 0.21
Total time: 0.000706 seconds
Sim of 1000 got probability of 0.24
Total time: 0.000270 seconds
Sim of 10000 got probability of 0.2534
Total time: 0.001210 seconds
Sim of 100000 got probability of 0.24841
Total time: 0.004434 seconds
Sim of 1000000 got probability of 0.250445
Total time: 0.042883 seconds
Sim of 5000000 got probability of 0.2500132
Total time: 0.206700 seconds


### Extra Credit: 

Extra credit: Instead of three dice, I now have six dice: d4, d6, d8, d10, d12 and d20. If I roll all six dice in “numerical” order, what is the probability I’ll get a strictly increasing sequence?

Pretty easy to implement with previous approach. Bumping max to 10 million runs.

In [7]:
# run larger sim: 
die_list = [4,6,8,10,12,20]
sim_list = [100, 1_000, 10_000, 100_000, 1_000_000, 5_000_000, 10_000_000]

for sim in sim_list:
    start = time.time()
    rolls = np.zeros(shape=(sim,len(die_list))) # build empty np array to store sim x total die

    # rolls over each die type
    for i, n in enumerate(die_list):
        rolls[:,i] = np.random.choice(n, sim) + 1 # do +1 to account for 0 index

    # use np.diff to create array of diffs of 0th -> 1st, 1st -> 2nd, etc.
    min_diff = np.min(np.diff(rolls), axis = 1) # axis = 1 indicates row
    print(f"Sim of {sim} got probability of {np.sum(min_diff > 0 )/sim}")
    print(f"Total time: {time.time() - start:2f} seconds")

Sim of 100 got probability of 0.0
Total time: 0.002777 seconds
Sim of 1000 got probability of 0.008
Total time: 0.000760 seconds
Sim of 10000 got probability of 0.0116
Total time: 0.001625 seconds
Sim of 100000 got probability of 0.01246
Total time: 0.007756 seconds
Sim of 1000000 got probability of 0.011893
Total time: 0.083576 seconds
Sim of 5000000 got probability of 0.0117234
Total time: 0.452892 seconds
Sim of 10000000 got probability of 0.0117774
Total time: 0.905959 seconds
